In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchaudio import datasets, transforms, info, load
from torch.utils.data import DataLoader, Dataset, random_split


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andradaolteanu/gtzan-dataset-music-genre-classification")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'gtzan-dataset-music-genre-classification' dataset.
Path to dataset files: /kaggle/input/gtzan-dataset-music-genre-classification


In [ ]:
data_path = '/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original'

In [ ]:
data_path

'/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original'

In [ ]:
import os
genres = os.listdir(data_path)
genres

['disco',
 'metal',
 'reggae',
 'blues',
 'rock',
 'classical',
 'jazz',
 'hiphop',
 'country',
 'pop']

In [ ]:
len(genres)

10

In [ ]:
label_to_index = {lab: ind for ind, lab in enumerate(genres)}

In [ ]:
label_to_index

{'disco': 0,
 'metal': 1,
 'reggae': 2,
 'blues': 3,
 'rock': 4,
 'classical': 5,
 'jazz': 6,
 'hiphop': 7,
 'country': 8,
 'pop': 9}

In [ ]:
transform = transforms.MelSpectrogram(
    sample_rate=22050,
    n_mels=64
)

In [ ]:
max_len = 500

In [ ]:
class GTZAN(Dataset):
  def __init__(self, root_path, transform, max_len):
    self.root_path = root_path
    self.transform = transform
    self.max_len = max_len
    self.audios = []

    for genre in os.listdir(root_path):
      genre_path = os.path.join(root_path, genre)
      for file in os.listdir(genre_path):
        if file.endswith('.wav'):
          file_path = os.path.join(genre_path, file)
          try:
            info(file_path)
            self.audios.append((file_path, genre))
          except Exception as e:
            print(f'Ошибка {e}')

  def __len__(self):
    return len(self.audios)

  def __getitem__(self, ind):
    file_path, genre = self.audios[ind]
    waveform, sr = load(file_path)


    if sr != 22050:
      resample = transforms.Resample(orig_freq=sr, new_freq=22050)
      waveform = resample(waveform)

    spec = self.transform(waveform).squeeze(0)

    if spec.shape[1] > self.max_len:
      spec = spec[:, :self.max_len]

    if spec.shape[1] < self.max_len:
      count_len = self.max_len - spec.shape[1]
      spec = F.pad(spec, (0, count_len))

    return spec, label_to_index[genre]

In [ ]:
dataset = GTZAN(data_path, transform, max_len)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size], generator=torch.Generator().manual_seed(42))


/tmp/ipython-input-3957819037.py:14: UserWarning: torchaudio._backend.utils.info has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  info(file_path)
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:20: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  s = torchaudio.io.StreamReader(s

Ошибка Failed to open the input "/kaggle/input/gtzan-dataset-music-genre-classification/Data/genres_original/jazz/jazz.00054.wav" (Invalid data found when processing input).


In [ ]:
train = DataLoader(train_data, batch_size=32, shuffle=True)
test = DataLoader(test_data, batch_size=32)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
class CheckAudio(nn.Module):
  def __init__(self):
    super().__init__()
    self.first = nn.Sequential(
        nn.Conv2d(1, 16, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.Conv2d(16, 32, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2),
        nn.AdaptiveAvgPool2d((8, 8)),
    )

    self.second = nn.Sequential(
        nn.Flatten(),
        nn.Linear(32 * 8 * 8, 128),
        nn.ReLU(),
        nn.Linear(128, 10)
    )


  def forward(self, x):
    x = x.unsqueeze(1)
    x = self.first(x)
    x = self.second(x)
    return x

In [ ]:
model = CheckAudio().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for epoch in range(15):
  model.train()
  total_loss = 0

  for x_batch, y_batch in train:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    y_pred = model(x_batch)
    loss = loss_fn(y_pred, y_batch)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  print(f'Эпоха: {epoch + 1}, Потери: {total_loss:.2f}')

/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be r

Эпоха: 1, Потери: 15.26
Эпоха: 2, Потери: 13.40
Эпоха: 3, Потери: 11.48
Эпоха: 4, Потери: 9.53
Эпоха: 5, Потери: 7.52
Эпоха: 6, Потери: 7.50
Эпоха: 7, Потери: 8.54
Эпоха: 8, Потери: 5.88
Эпоха: 9, Потери: 5.84
Эпоха: 10, Потери: 7.13
Эпоха: 11, Потери: 7.41
Эпоха: 12, Потери: 5.70
Эпоха: 13, Потери: 6.29
Эпоха: 14, Потери: 3.64
Эпоха: 15, Потери: 3.72


In [ ]:
model.eval()
correct, total = 0, 0
with torch.no_grad():
  for x_batch, y_batch in test:
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)
    y_pred = model(x_batch)
    pred = torch.argmax(y_pred, dim=1)

    correct += (pred == y_batch).sum().item()
    total += y_batch.size(0)

accuracy = correct * 100 / total
print(f'Точность: {accuracy}')

Точность: 56.0


In [ ]:
torch.save(model.state_dict(), 'gtzan_model.pth')
torch.save(genres, 'gtzan_labels.pth')